#### <h1> <center> ENSF 592 Programming Fundamentals for Data Engineers </center></h1>
<h2> <center> Assignment 6: Pandas (5 points)</center></h2>

<center>
<div class="alert alert-block alert-info">
Due: Wed Nov 6 12am (midnight, Tue to Wed). To be submitted on D2L.
</div></center>




Edit this file and write your solutions to the problems in sections specified with `# Your solution goes here`. Test your code and when you are ready, download this notebook as an `.ipynb` file and submit it to D2L. To get this file, in Jupyter notebook you can go to File -> Download as -> Notebook(.ipynb)

In [2]:
import numpy as np
import pandas as pd

## 1. Pandas Health Report (2 pts)

Generate the same report as in Assignment05 PartII, but this time use Pandas dataframe instead of `Table` class. 

In [3]:

df = pd.read_csv('data.csv', index_col=False, header=0,na_values="?")

# report 1
print("Report1\n \tthe average age of patients are: ",df[['age']].mean())

# report2
print("\nReport2")

    # def rangeSelect(self,column:str,begin:int,end:int):
    #     index = self.data.columns.get_loc(column)
    #     x = self.data.values
    #     df = pd.DataFrame(x)
    #     df.dropna(subset=[index])
    #     return df.loc[(df[index] >= begin) & (df[index] < end)].values
for i in range(20,60,10):
    index = df.columns.get_loc('age')
    df2 = pd.DataFrame(df.values)
    df2.dropna(subset =[index])
    print("\tfor people with age in range of", i,i+10,"average CHOL is: ",np.nanmean(df2.loc[(df2[index] >= i) & (df2[index] < i+10)].values[:,4]))

# report3
#  def percentageSelect(self,column:str,perc:float,index:int):
     
col_index = df.columns.get_loc('chol')
df3 = pd.DataFrame(df.values)
df3.dropna(subset=[col_index])
df3 = df3.sort_values(col_index)
num_elements = df3.shape[0]

print("\nReport3")
print("\tfor patients with lowest 30% of chol, average trestbps is: ",np.nanmean(df3.head(int(num_elements * 30 / 100)).values[:,3]))
print("\tfor patients with highest 30% of chol, average trestbps is: ",np.nanmean(df3.tail(int(num_elements * 30 / 100)).values[:,3])) 

# report4
print("\nReport4")


index = df.columns.get_loc('num')
df4 = pd.DataFrame(df.values)
df4.dropna(subset=[index])
res = df4.loc[df4[index] == 1.0].values

men = sum(res[:,1])/res.shape[0]
print("\t{0:.2f}% of patients diagnosed with heart decease are men and ".format( men ),"{0:.2f}% of them are women".format( 1-men  ))


Report1
 	the average age of patients are:  age    47.767918
dtype: float64

Report2
	for people with age in range of 20 30 average CHOL is:  187.5
	for people with age in range of 30 40 average CHOL is:  239.6304347826087
	for people with age in range of 40 50 average CHOL is:  250.13131313131314
	for people with age in range of 50 60 average CHOL is:  256.2105263157895

Report3
	for patients with lowest 30% of chol, average trestbps is:  133.22988505747125
	for patients with highest 30% of chol, average trestbps is:  136.09302325581396

Report4
	0.89% of patients diagnosed with heart decease are men and  0.11% of them are women


## 2. Historical Weather Data (1.5pts)

Chapter 5 in https://github.com/jvns/pandas-cookbook  
shows how to download historical weather data for Canadian weather stations in a csv file.

### Background
In the jupyter notebook, we find landing page:  
https://climate.weather.gc.ca

with a link to interactively search for data:  
https://climate.weather.gc.ca/historical_data/search_historic_data_e.html

searching for 'Calgary' we find for example Calgary International Airport and can get data for a past date, either hourly or daily data. There is a link to 'Get more data':  
https://drive.google.com/drive/folders/1WJCDEU34c60IfOnG4rv5EPZ4IhhW9vZH

The Readme.pdf explains the API and all station IDs needed to request the data are listed in Station Inventory EN.xlsx

Calgary International Airport has ID 2205 for data up to Jul 2012 and 50430 for data after Jul 2012

### Retrieving data
As explained in the pandas cookbook notebook, the following code loads data into a pandas dataframe:
```python
# url string with variables station, year and month
url_template = "https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID={station}&Year={year}&Month={month}&Day=14&timeframe=1&submit=Download+Data"

# replace variables in string with actual values
url = url_template.format(station=2205, year=2001, month=5)

# read data into dataframe, use headers and set Date/Time column as index
weather_data = pd.read_csv(url, index_col='Date/Time', parse_dates=True)

# replace the degree symbol in the column names
weather_data.columns = [col.replace('\xb0', '') for col in weather_data.columns]
```

Note that replacing `timeframe=1` with `timeframe=2` in the `url_template` will retrieve daily data rather than hourly data and a full year is returned (month is ignored).

### Tasks
1. Write a function `download_weather_data` that takes station, year and month as arguments and can download either hourly or daily data (daily data by default). Data is returned in a DataFrame with degree symbol removed in column names.

2. Compare the amount of snow fall in 1918 and 2018. In which year was the total snowfall higher?

3. (Optional) Find another interesting fact in the data, e.g. how many days in a certain year snow was on the ground, hottest summer in the past 5 years, etc.

4. (Optional) Download the 'Staion Inventory EN.xlsx' and write a function that allows extracting station IDs by supplying station names.

In [111]:
# Your solution goes here

def download_weather_data(station, year, month=1, daily=True):
    """ returns a DataFrame with weather data from climate.weather.gc.ca"""
    time_frame = 2 if daily else 1
    url_template = f"https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID={station}&Year={year}&Month={month}&Day=14&timeframe={time_frame}&submit=Download+Data"
    weather_data = pd.read_csv(url_template, index_col='Date/Time', parse_dates=True)
    weather_data.columns = [col.replace('\xb0', '') for col in weather_data.columns]
    return weather_data

 
data = download_weather_data(station=2205, year=1918, month=1, daily=True)
total_snow = np.nansum(data[['Total Snow (cm)' ]].values)



data2 = download_weather_data(station=50430, year=2018, month=1, daily=True)
total_snow2 = np.nansum(data2[['Total Snow (cm)' ]].values)

print(f"Snowfall in 2018 was {total_snow2}, in 1918 total snowfall was {total_snow},\n"
      f" but then again the station moved so maybe that was a factor.")

Snowfall in 2018 was 213.8, in 1918 total snowfall was 70.0,
 but then again the station moved so maybe that was a factor.


## 3. WikiQuotes (1.5pts)

Data set from Kaggle: https://www.kaggle.com/fantop/wikiquote-short-english-quotes/data

### Tasks
1. Read the json file `quotes-100-en.json` into a dictionary using the json module
2. Write a function `search_quotes` that takes a string as argument, searches the keys in the quotes dictionary for items containing the search string and prints the matching name (key) and quotes.
3. With the function in 2. you can verify that there are quotes for 'Einstein' and 'Feynman'. Ranking by number of quotes, who of the two is ranked higher and what is their respective rank? To answer this question, build a DataFrame with quotes keys in one column and number of quotes in a second column. With this Dataframe:
    - Sorting the DataFrame by number of quotes, 
    - resetting the index 
    - and finding both individuals the new index shows the rank.
4. (optional) Are there any quotes that are equal or close? Look up 'Jaccard index for text similarity', use this index to:
    - calculate similarities between quotes
    - and define a threshold for calling two quotes 'close'
    - output quotes that are close.

In [51]:
import json
import pandas as pd
import numpy as np
# help(json)

In [112]:
import json
import requests
import os
from pprint import pprint



# Your solution goes here

def search_quotes(name, quotes):
    """Prints quotes from quotes dictionary containing name in the key."""
    for key, value in quotes.items():
        if name in key:
            print(key+ "\n")
            for q in quotes[key]:
                print("\t"+ q + "\n")

def score_quotes(quotes):
    score = {}
    for key, values in quotes.items():
        score[key] = len(values)
    return score
        # if (obj[0] == name):
        #     for q in obj[0]
        #         pprint(q)

with open('quotes-100-en.json') as json_file:
    data = json.load(json_file)
# df = pd.DataFrame(data)

# pprint(data)
search_quotes("Einstein", data)
search_quotes("Feynman", data)
score = score_quotes(data)


names = ['origin', 'count']
array = np.array(list(score.items()))
df = pd.DataFrame(array,columns=names).sort_values(by=['count'], ascending=False)
df['index'] = np.arange(len(df))
# print(score)
sum1 = 0
print(f"Associated with 'Einstein' are:")
for line in df[df['origin'].str.contains("Einstein")].values:
    sum1 += int(line[1])
    print(f"\t{line[0]}, rank {line[2]}, count {line[1]}")
print(f"Counts attributed to Einstein were: {sum1}")

sum2 =0
print(f"\nAssociated with 'Feynman' are:")
for line in df[df['origin'].str.contains("Feynman")].values:
    sum2 += int(line[1])
    print(f"\t{line[0]}, rank {line[2]}, count {line[1]}")
print(f"Counts attributed to Feynman were: {sum2}")
    # if "Einstein" in key:
    #     print(key + " ")
    #     print(score[key])

Albert Einstein

	The mass of a body is a measure of its energy content.

	Sometimes one pays most for the things one gets for nothing.

	I refuse to make money out of my science. My laurel is not for sale like so many bales of cotton.

	Life is like riding a bicycle. To keep your balance you must keep moving.

	The really good music, whether of the East or of the West, cannot be analyzed.

	I never think of the future. It comes soon enough.

	Only a life lived for others is a life worthwhile.

	Force always attracts men of low morality.

	All of science is nothing more than the refinement of everyday thinking.

	One may say "the eternal mystery of the world is its comprehensibility."

	Perfection of means and confusion of goals seem—in my opinion—to characterize our age.

	Why is it nobody understands me and everybody likes me?

	That is simple, my friend. It is because Politics is more difficult than physics. 

	It is easier to denature plutonium than it is to denature the evil spiri

## 4. Some other data (optional)

Find a dataset or two on one of these repositories:

https://www.kaggle.com/datasets

https://archive.ics.uci.edu/ml/index.php

https://github.com/iamaziz/PyDataset


Load it into Pandas and report on something interesting.